<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#aggregate-and-filter" data-toc-modified-id="aggregate-and-filter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>aggregate and filter</a></span></li></ul></div>

# aggregate and filter

- https://stackoverflow.com/questions/59931770/sum-of-array-elements-depending-on-value-condition-pyspark

In [1]:
import numpy as np
import pandas as pd

# pyspark
import pyspark
spark = pyspark.sql.SparkSession.builder.appName('app').getOrCreate()

# sql
from pyspark.sql.functions import col as _col
from pyspark.sql.functions import udf

# @udf("integer") def myfunc(x,y): return x - y
# stddev format_number date_format, dayofyear, when
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import (mean as _mean, min as _min,
                                   max as _max, avg as _avg,
                                   when as _when
                                  )

from pyspark.sql.types import (StructField,StringType,
                               IntegerType, FloatType,
                               DoubleType,StructType)

from pyspark import SparkConf, SparkContext, SQLContext

sc = spark.sparkContext
sqlContext = SQLContext(sc) 
sqc = sqlContext
# spark_df = sqlContext.createDataFrame(pandas_df)

In [4]:
from pyspark.sql.functions import expr

In [5]:
sdf = spark.createDataFrame([(1,[0.2, 2.1, 3., 4., 3., 0.5]),
                             (2,[7., 0.3, 0.3, 8., 2.,])],                                        ['id','column'])

sdf.toPandas()

,id,column
0,1,"[0.2, 2.1, 3.0, 4.0, 3.0, 0.5]"
1,2,"[7.0, 0.3, 0.3, 8.0, 2.0]"


In [8]:
(sdf
.withColumn("column<2",
  expr("""aggregate(filter(column, x -> x < 2), 0D,
  (x, acc) -> acc + x)"""))
 
.withColumn("column>2",
  expr("""aggregate(filter(column, x -> x > 2), 0D,
  (x, acc) -> acc + x)"""))
 
.withColumn("column=2",
  expr("""aggregate(filter(column, x -> x == 2), 0D,
  (x, acc) -> acc + x)"""))

).toPandas()

,id,column,column<2,column>2,column=2
0,1,"[0.2, 2.1, 3.0, 4.0, 3.0, 0.5]",0.7,12.1,0.0
1,2,"[7.0, 0.3, 0.3, 8.0, 2.0]",0.6,15.0,2.0
